In [3]:
import pandas as pd
import eikon as ek
import numpy as np
from IPython.display import display, HTML

In [4]:
# display(HTML("<style>.container { width:100% !important; }</style>"))
# display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [ ]:
class Eikon:
    
    def __init__(self, path_api_key: str = r'C:\Users\serge\OneDrive\Documents'):
        
        ek_api_key = pd.read_csv(fr'{path_apikeys}\apikeys.csv', names=['api', 'key'], index_col=0)
        ek.set_app_key(ek_api_key.loc['reuters'].values[0])
        
        self.api = ek
    
    def get_timeseries(self, rics: list, field: str = 'TR.PriceClose', date_field: str = 'TR.PriceClose.calcdate', params: dict = None):
        if not isinstance(rics, list):
            raise AttributeError('rics parameter must be list, if it is single RIC, convert to list first')
        data_dict = {}
        for ric in indices_rics:
            try:
                print(ric)
                data_dict[ric] = ek.get_data(ric, [field, date_field] , parameters=params)[0]
                data_dict[ric].loc[:, 'Calc Date'] = data_dict[ric].loc[:, 'Calc Date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
                data_dict[ric].loc[:, 'Month'] = data_dict[ric].loc[:, 'Calc Date'].apply(lambda x: x.month)
                data_dict[ric].loc[:, 'Year'] = data_dict[ric].loc[:, 'Calc Date'].apply(lambda x: x.year)
                data_dict[ric].loc[:, 'DailyLogReturns'] = np.log(data_dict[ric]['Price Close'] / data_dict[ric]['Price Close'].shift(1))
                print('Done')
            except Exception as e:
                print(e)
        
#         self.data = pd.DataFrame.from_dict(data_dict, orient='index')
          self.data = data_dict
        
    
    def get_index_constituents(self, index: str = '.GDAXI', date: str = '20230321'):
        
        all_rics = []
        t0 = time.time()
        index = list(index)
        for i in range(len(index)):
            try:
                temp_rics, err = self.api.get_data(index[i], ['TR.IndexConstituentRIC' , 'TR.IndexConstituentName'], {'SDate': date})
            # all_rics, err = ek.get_data(indices_rics[2], ['TR.IndexConstituentRIC' , 'TR.IndexConstituentName'])
                print(f'Retrieved {time.time() - t0}')
            except Exception as e:
                print(err)
            if err is None:
                all_rics.append(temp_rics['Constituent RIC'].to_list())
                
        return all_rics

In [ ]:
class FamaFrenchFactors:
    
    def __init__(self, path_data: str = r'C:\Users\serge\IdeaProjects\portfolio_manager\data\fama-french-factors'):
        self.path = path_data
        
    def get_data(self, file: str):
        header_row = 2
        index_row = 0
        df_factors = pd.read_csv(filepath_or_buffer=fr'{self.path}\\{file}',
                                   header=header_row,
                                   index_col=index_row,
                                   parse_dates=True, 
                                   usecols=range(7))
        #                            nrows=(end_row-8))

        df_factors.replace({-99.99:np.nan}, inplace=True)
        df_factors = df_factors / 100
        return df_factors
    
    